# Using submodels in PyBaMM
In this notebook we show how to modify existing models by swapping out submodels, and how to build your own model from scratch using exisitng submodels. To see all of the models and submodels available in PyBaMM, please take a look at the documentation [here](https://pybamm.readthedocs.io/en/latest/source/models/index.html).

## Changing a submodel in an exisiting battery model
PyBaMM is designed to be a flexible modelling package that allows users to easily compare different models and numerical techniques within a common framework. Battery models within PyBaMM are built up using a number of submodels that describe different physics included within the model, such as mass conservation in the electrolyte or charge conservation in the solid. For ease of use, a number of popular battery models are pre-configured in PyBaMM. As an example, we look at the Single Particle Model (for more information see [here](./models/SPM.ipynb)). First we import PyBaMM and any other packages we need

In [1]:
import pybamm
import os
import numpy as np
import matplotlib.pyplot as plt
os.chdir(pybamm.__path__[0]+'/..')

Then we load the SPM

In [2]:
model = pybamm.lithium_ion.SPM()

We can look at the submodels that make up the SPM by accessing `model.submodels`, which is a dictionary of the submodel's name (i.e. the physics it represents) and the submodel that is selected

In [3]:
for name, submodel in model.submodels.items():
    print(name, submodel)

external circuit <pybamm.models.submodels.external_circuit.current_control_external_circuit.CurrentControl object at 0x7f36b3a88590>
porosity <pybamm.models.submodels.porosity.constant_porosity.Constant object at 0x7f36afe4d0d0>
electrolyte tortuosity <pybamm.models.submodels.tortuosity.bruggeman_tortuosity.Bruggeman object at 0x7f36b537e0d0>
electrode tortuosity <pybamm.models.submodels.tortuosity.bruggeman_tortuosity.Bruggeman object at 0x7f36f0ed3dd0>
through-cell convection <pybamm.models.submodels.convection.through_cell.no_convection.NoConvection object at 0x7f36f0ed3790>
transverse convection <pybamm.models.submodels.convection.transverse.no_convection.NoConvection object at 0x7f36afdef7d0>
negative interface <pybamm.models.submodels.interface.inverse_kinetics.inverse_butler_volmer.InverseButlerVolmer object at 0x7f36afdef810>
positive interface <pybamm.models.submodels.interface.inverse_kinetics.inverse_butler_volmer.InverseButlerVolmer object at 0x7f36afdef8d0>
negative interf

When you load a model in PyBaMM it builds by default. Building the model sets all of the model variables and sets up any variables which are coupled between different submodels: this is the process which couples the submodels together and allows one submodel to access variables from another. If you would like to swap out a submodel in an exisitng battery model you need to load it without building it by passing the keyword `build=False`

In [4]:
model = pybamm.lithium_ion.SPM(build=False)

This collects all of the submodels which make up the SPM, but doesn't build the model. Now you are free to swap out one submodel for another. For instance, you may want to assume that diffusion within the negative particles is infinitely fast, so that the PDE describing diffusion is replaced with an ODE for the uniform particle concentration. To change a submodel you simply update the dictionary entry

In [5]:
model.submodels["negative particle"] = pybamm.particle.FastSingleParticle(model.param, "Negative")
model.submodels["positive particle"] = pybamm.particle.FastSingleParticle(model.param, "Positive")
model.submodels["particle cracking"] = pybamm.particle_cracking.CrackPropagation(model.param, "Negative")

Now if we look at the submodels again we see that the model for the negative particle has been changed

In [6]:
for name, submodel in model.submodels.items():
    print(name, submodel)

external circuit <pybamm.models.submodels.external_circuit.current_control_external_circuit.CurrentControl object at 0x7f36af618310>
porosity <pybamm.models.submodels.porosity.constant_porosity.Constant object at 0x7f36af618590>
electrolyte tortuosity <pybamm.models.submodels.tortuosity.bruggeman_tortuosity.Bruggeman object at 0x7f36af618690>
electrode tortuosity <pybamm.models.submodels.tortuosity.bruggeman_tortuosity.Bruggeman object at 0x7f36af618790>
through-cell convection <pybamm.models.submodels.convection.through_cell.no_convection.NoConvection object at 0x7f36af618890>
transverse convection <pybamm.models.submodels.convection.transverse.no_convection.NoConvection object at 0x7f36af618990>
negative interface <pybamm.models.submodels.interface.inverse_kinetics.inverse_butler_volmer.InverseButlerVolmer object at 0x7f36af618a90>
positive interface <pybamm.models.submodels.interface.inverse_kinetics.inverse_butler_volmer.InverseButlerVolmer object at 0x7f36af618b50>
negative interf

Building the model also sets up the equations, boundary and initial conditions for the model. For example, if we look at `model.rhs` before building we see that it is empty 

In [7]:
model.rhs

{}

If we try to use this empty model, PyBaMM will give an error. So, before proceeding we must build the model

In [8]:
model.build_model()

Now if we look at `model.rhs` we see that it contains an entry relating to the concentration in each particle, as expected for the SPM

In [9]:
model.rhs

{Variable(-0x7eab5eaefec3a80e, Discharge capacity [A.h], children=[], domain=[], auxiliary_domains={}): Division(-0x36eb5495f1c87f5d, /, children=['Current function [A] * 96485.33212 * Maximum concentration in negative electrode [mol.m-3] * Negative electrode thickness [m] + Separator thickness [m] + Positive electrode thickness [m] / function (absolute)', '3600.0'], domain=[], auxiliary_domains={}),
 Variable(0x76812dad1f41019a, X-averaged negative particle surface concentration, children=[], domain=['current collector'], auxiliary_domains={}): Division(-0x7a6f5c59b0f94dd1, /, children=["-3.0 * integral dx_n ['negative electrode'](broadcast(broadcast(Current function [A] / Typical current [A] * function (sign)) / Negative electrode thickness [m] / Negative electrode thickness [m] + Separator thickness [m] + Positive electrode thickness [m]) - broadcast(0.0) + broadcast(0.0)) / Negative electrode thickness [m] / Negative electrode thickness [m] + Separator thickness [m] + Positive elec

Now the model can be processed and solved in the usual way, and we still have access to model defaults such as the default geometry and default spatial methods

In [10]:
# create geometry
geometry = model.default_geometry

# load parameter values and process model and geometry
param = model.default_parameter_values
param.process_model(model)
param.process_geometry(geometry)

# set mesh
mesh = pybamm.Mesh(geometry, model.default_submesh_types, model.default_var_pts)

# discretise model
disc = pybamm.Discretisation(mesh, model.default_spatial_methods)
disc.process_model(model)

# solve model
t_eval = np.linspace(0, 3600, 100)
solution = model.default_solver.solve(model, t_eval)

# extract voltage
stress_t_n_surf = solution['Negative particle surface tangential stress [Pa]']

# plot
plt.plot(solution["Time [h]"](solution.t), stress_t_n_surf(solution.t))
plt.xlabel(r'$t$')
plt.ylabel('Negative particle surface tangential stress')
plt.show()

/home/aishine/GEM/PyBaMM/pybamm/expression_tree/binary_operators.py:533: RuntimeWarning: divide by zero encountered in true_divide
  return left / right
/home/aishine/GEM/PyBaMM/pybamm/expression_tree/binary_operators.py:417: RuntimeWarning: invalid value encountered in multiply
  return left * right


AssertionError: 
                rhs and initial_conditions must have the same shape after discretisation
                but rhs.shape = (20, 1) and initial_conditions.shape = (10, 1) for variable 'Negative particle crack length'.
                

In [ ]:
param.process_model(model)

## Building a custom model from submodels
Instead of editing a pre-exisitng model, you may wish to build your own model from scratch by combining exisitng submodels of you choice. In this section, we build a Single Particle Model in which the diffusion is assumed infinitely fast in both particles. 

To begin, we load a base lithium-ion model. This sets up the basic model structure behind the scenes, and also sets the default paramaters to be those corresponding to a lithium-ion battery. Note that the base model does not select any default submodels, so there is no need to pass `build=False`.

In [ ]:
model = pybamm.lithium_ion.BaseModel()

Submodels can be added to the `model.submodels` dictionary in the same way that we changed the submodels earlier. 

We use the simplest model for the external circuit, which is the "current control" submodel

In [ ]:
model.submodels["external circuit"] = pybamm.external_circuit.CurrentControl(model.param)

We want to build a 1D model, so select the `Uniform` current collector model (if the current collectors are behaving uniformly, then a 1D model is appropriate). We also want the model to be isothermal, so slect the thermal model accordingly. 

In [ ]:
model.submodels["current collector"] = pybamm.current_collector.Uniform(model.param)
model.submodels["thermal"] = pybamm.thermal.isothermal.Isothermal(model.param)
model.submodels["particle cracking"] = pybamm.particle_cracking.particle_cracking(model.param)

We assume that the current density varies linearly in the electrodes. This corresponds the the leading-order terms in Ohm's law in the limit in which the SPM is derived in [1]

In [ ]:
model.submodels["negative electrode"] = pybamm.electrode.ohm.LeadingOrder(
    model.param, "Negative"
)
model.submodels["positive electrode"] = pybamm.electrode.ohm.LeadingOrder(
    model.param, "Positive"
)

We assume that diffusion is infinitely fast in both the negative and positive particles 

In [ ]:
model.submodels["negative particle"] = pybamm.particle.FastSingleParticle(
    model.param, "Negative"
)
model.submodels["positive particle"] = pybamm.particle.FastSingleParticle(
    model.param, "Positive"
)

In the Single Particle Model, the overpotential can be obtianed by inverting the Butler-Volmer relation, so we choose the `InverseButlerVolmer` submodel for the interface, with the "main" lithium-ion reaction

In [ ]:
model.submodels[
    "negative interface"
] = pybamm.interface.InverseButlerVolmer(model.param, "Negative", "lithium-ion main")
model.submodels[
    "positive interface"
] = pybamm.interface.InverseButlerVolmer(model.param, "Positive", "lithium-ion main")


Finally, for the electrolyte we assume that diffusion is infinitely fast so that the concentration is uniform, and also use the leading-order model for charge conservation, which leads to a linear variation in ionic current in the electrodes

In [ ]:
model.submodels["electrolyte diffusion"] = pybamm.electrolyte_diffusion.ConstantConcentration(
    model.param
)
model.submodels["electrolyte conductivity"] = pybamm.electrolyte_conductivity.LeadingOrder(
    model.param
)

Now that we have set all of the submodels we can build the model

In [ ]:
model.build_model()

The base lithium-ion model doesn't come with a default geometry, so we choose a 1D macroscopic geometry for the electrodes and separator, and 1D microscopic geometry for the particles (i.e. the geometry of the standard Newman model [2])

In [ ]:
geometry = pybamm.Geometry("1D macro", "1D micro")

The base model does come with defaults for the parameters, spatial methods and submeshes, so we can now proceed to solve the model in the usual way 

In [ ]:
# process model and geometry
param = model.default_parameter_values
param.process_model(model)
param.process_geometry(geometry)

# set mesh
mesh = pybamm.Mesh(geometry, model.default_submesh_types, model.default_var_pts)

# discretise model
disc = pybamm.Discretisation(mesh, model.default_spatial_methods)
disc.process_model(model)

# solve model
t_eval = np.linspace(0, 3600, 100)
solver = pybamm.ScipySolver()
solution = solver.solve(model, t_eval)

# extract voltage
voltage = solution['Terminal voltage [V]']

# plot
plt.plot(solution["Time [h]"](solution.t), voltage(solution.t))
plt.xlabel(r'$t$')
plt.ylabel('Terminal voltage')
plt.show()

## References
[1] SG Marquis, V Sulzer, R Timms, CP Please and SJ Chapman. “An asymptotion derivation of a single particle model with electrolyte”. In: arXiv preprint arXiv:1905.12553 (2019).

[2] M Doyle, TF Fuller and J Newman. "Modeling of galvanostatic charge and discharge of the lithium/polymer/insertion cell." Journal of the Electrochemical Society 140.6 (1993): 1526-1533.